# <center>Data Analysis 3 - Assignment 1<center>
    
<center>Created by Zsófia Rebeka Katona</center>


Build four predictive models using linear regression for earnings per hour.
1. Models: the target variable is earnings per hour, all others would be predictors

In [1]:
# Importing the required libraries
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from mizani.formatters import percent_format
import os
from plotnine import *
import numpy as np
import sys
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from stargazer.stargazer import Stargazer
from statsmodels.tools.eval_measures import mse,rmse

In [2]:
# Importing the prewritten helper functions
from py_helper_functions import *

ModuleNotFoundError: No module named 'py_helper_functions'

In [ ]:
# Reading the data
data = pd.read_csv("morg-2014-emp.csv")

# Checking if the data was correctly loaded
data.head()

In [ ]:
data.info()

### Exploratory Data Analysis

In [ ]:
data.columns

In [ ]:
# Filtering for Human resource managers (0136) and Human resource workers (630)
data = data[data['occ2012'].isin([136, 630])]
data

### Creating variables:
- earnings per hour
- log earnings per hour
- female as a dummy variable
- age
- age squared
- level of education

In [ ]:
# Creating new variable: earnings per hour
data["earnphour"] = data["earnwke"] / data["uhours"]
data["lnearnphour"] = np.log(data["earnphour"])
data["female"] = (data["sex"] == 2)
data["agesq"] = np.power(data["age"], 2)
data["edu"] = data["grade92"]

In [ ]:
# Grouping educational levels and creating dummies
# 'hsedu' referring to high school education
# "higheredu" referring to higher education, including Bachelors, Masters, PhD and Professional degrees
data["hsedu"] = (data["grade92"] == 39).astype(int)
data["higheredu"] = (data["grade92"].isin([40, 41 ,42 , 43, 44, 45, 46]).astype(int))

In [ ]:
# Checking for the quality of the distribution and sufficient variation
data.earnphour.hist(bins=100)

In [ ]:
# Checking the log earnings per hour distribution
data.lnearnphour.hist(bins=100)

In [ ]:
# Checking if we have 0 or missing values
data.sort_values(by = 'earnphour')

# There are no values = 0

In [ ]:
# Checking for NaN in the 'earnwke' column
data.loc[data["earnphour"].isna()]

# There are no missing values

#### Model 1. - Regression of earnings per hour on women

In [ ]:
# Creating the regression and checking the summary of regression
reg1 = smf.ols(formula="lnearnphour~female", data=data).fit(cov_type="HC1")
reg1.summary()

In [ ]:
# Regression summary with HC' type of robust error in another form
print(reg1.get_robustcov_results(cov_type='HC1').summary())

In [ ]:
# Checking the BIC of regression 1 (Model 1)
reg1.bic

#### Model 2.: Expanding the model by adding the age variable

In [ ]:
# Creating regression for Model 2
reg2 = smf.ols(formula="lnearnphour~female + age", data=data).fit(cov_type="HC1")
reg2.summary()

In [ ]:
# Checking the BIC of regression 2 (Model 2)
reg2.bic

In [ ]:
stargazer = Stargazer([reg1, reg2])
stargazer.covariate_order(["female[T.True]", "Intercept"])
stargazer.rename_covariates({"Intercept": "Constant"})
stargazer

#### Model 3. Expanding the model by adding the age squared variable

In [ ]:
# Creating the regression of Model 3
reg3 = smf.ols(formula="lnearnphour~ female + age + agesq", data=data).fit(cov_type="HC1")
reg3.summary()

In [ ]:
# Checking the BIC of regression 3 (Model 3)
reg3.bic

In [ ]:
# Creating the regression of Model 4
reg4 = smf.ols(formula="lnearnphour~ female + age + agesq + higheredu", data=data).fit(cov_type="HC1")
reg4.summary()

In [ ]:
# Summarizing the regression results with HC1 robust standard error
models = [reg1, reg2, reg3, reg4]
robustcov_results=[]

for i, model in enumerate(models):
    result=model.get_robustcov_results(cov_type='HC1').summary()
    robustcov_results.append(result)
    print()
    print(f'Regression: reg{i+1}')
    print(result)

In [ ]:
bic = [round(x.bic, 2) for x in [reg1,reg2,reg3,reg4]]
sg = stargazer.Stargazer([reg1,reg2,reg3,reg4])
sg.add_line('BIC', bic, location=stargazer.LineLocation.FOOTER_BOTTOM)
sg

#### Model 2.: Linear Regression with cross-validation

In [ ]:
# Importing the package for cross-validation
from sklearn.model_selection import KFold
# Splitting the data into 4 categories, assuming that the data is randomly ordered
k = KFold(n_splits=4, shuffle=False, random_state=None)

In [ ]:
# Defining the cross-validation 
def cv_reg(formula, data, kfold, robustse=None):
    regression_list = []
    predicts_on_test = []
    rsquared = []
    rmse_list = []

    # Calculating OLS for each fold

    for train_index, test_index in k.split(data):
        # print("TRAIN:", train_index, "TEST:", test_index)
        data_train, data_test = data.iloc[train_index, :], data.iloc[test_index, :]
        if robustse is None:
            model = smf.ols(formula, data=data_train).fit()
        else:
            model = smf.ols(formula, data=data_train).fit(cov_type=robustse)
        regression_list += [model]
        predicts_on_test += [model.predict(data_test)]
        rsquared += [model.rsquared]
        rmse_list += [rmse(data_train[formula.split("~")[0]], model.predict())]

    return {
        "regressions": regression_list,
        "test_predict": predicts_on_test,
        "r2": rsquared,
        "rmse": rmse_list,
    }


def summarize_cv(cvlist, stat="rmse"):
    result = pd.DataFrame(
        {"Model" + str(x + 1): cvlist[x][stat] for x in range(len(cv_list))}
    )
    result["Resample"] = ["Fold" + str(x + 1) for x in range(len(cvlist[0]["rmse"]))]
    result = result.set_index("Resample")
    result = pd.concat([result, pd.DataFrame(result.mean(), columns=["Average"]).T])
    return result

In [ ]:
#reg1 = smf.ols(formula="lnearnphour~female", data=data).fit(cov_type="HC1")
#reg2 = smf.ols(formula="lnearnphour~female + age", data=data).fit(cov_type="HC1")
#reg3 = smf.ols(formula="lnearnphour~ female + age + agesq", data=data).fit(cov_type="HC1")
#reg4 = smf.ols(formula="lnearnphour~ female + age + agesq + higheredu", data=data).fit(cov_type="HC1")

cv1 = cv_reg("lnearnphour~female", data, k, "HC0")
cv2 = cv_reg("lnearnphour~female + age", data, k, "HC0")
cv3 = cv_reg("lnearnphour~ female + age + agesq", data, k, "HC0")
cv4 = cv_reg("lnearnphour~ female + age + agesq + higheredu", data, k, "HC0")

cv_list = [cv1, cv2, cv3, cv4]

In [ ]:
# Cross-validation summary table
summarize_cv(cv_list)

In [ ]:
# Checking the residual values for Model 1
reg1.resid.describe()

In [ ]:
# Getting the predictions using regression 1 and extracting its summary statistics
p1=reg1.get_prediction(data).summary_frame()
p1

In [ ]:
# Getting the basic metrics of fitted values
(reg3.fittedvalues-data.lnearnphour).describe()

In [ ]:
# Getting the predictions using regression 3 and extracting its summary statistics
p2=reg3.get_prediction(data).summary_frame()
p2

In [ ]:
# Getting the root squared mean error (RMSE) of Model 3
rmse(reg3.fittedvalues,data.lnearnphour)

In [ ]:
# Creating a DataFrame for the predicted values and the 95% prediciton intervals 
pd.DataFrame(
    {
        " ": ["Predicted", "PI_low(95%)", "PI_high(95%)"],
        "Model1": p1[["mean", "obs_ci_lower", "obs_ci_upper"]].values.tolist()[0],
        "Model3": p2[["mean", "obs_ci_lower", "obs_ci_upper"]].values.tolist()[0],
    }
).set_index(" ")

In [ ]:
# Summary of the predictions with 80% prediction intervals
p1=reg1.get_prediction(data).summary_frame(alpha=0.2)
p2=reg3.get_prediction(data).summary_frame(alpha=0.2)

pd.DataFrame(
    {
        " ": ["Predicted", "PI_low(80%)", "PI_high(80%)"],
        "Model1": p1[["mean", "obs_ci_lower", "obs_ci_upper"]].values.tolist()[0],
        "Model3": p2[["mean", "obs_ci_lower", "obs_ci_upper"]].values.tolist()[0],
    }
).set_index(" ")